In [258]:
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import time

In [153]:
target_url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

In [24]:
headers = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [25]:
webpage=requests.get(target_url,headers=headers)

In [26]:
webpage

<Response [200]>

In [28]:
soup = BeautifulSoup(webpage.content,'html.parser')

#### On visiting product page using product URL

In [232]:
#Function to extract product title
def get_title(soup):
    try:
        title=soup.find("span", attrs={'id':'productTitle'})
        
        #Inner NavigatableString Object
        title_value = title.text
        
        # Title as a string value
        title_string = title_value.strip()
    except:
        title_string =""
    
    return title_string

In [246]:
#Function to extract ASIN
def get_ASIN(soup):
    asins=[]
    for product in soup.find_all("div", {"data-asin": True}):
        asin = product["data-asin"]
        if asin!="":
            asins.append(asin)
    return asins[0:29]

In [252]:
#Function to extract manufacturer
def get_manufacturer(soup):
    manfs=[]
    for product in soup.find_all("div", {"data-manufacturer": True}):
        manf = product["data-manufacturer"]
        if manf!="":
            manfs.append(manf)
    return manfs[0:29]

### Main driver function

In [262]:
if __name__ == '__main__':
    
    #given URL to target webpage
    url = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"
    
    #headers
    headers = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
    
    #
    #webpage=requests.get(url,headers=headers)
    
    soup = BeautifulSoup(webpage.content,'html.parser')
    
    links = soup.find_all("a", attrs={'class' :"a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" })
    
    product_urls_list =[]
    product_names_list =[]
    product_prices_list =[]
    product_ratings_list =[]
    product_reviews_list =[]
    
    #Product URL and Name
    for link in links:
        product_urls_list.append("https://www.amazon.in"+link.get('href'))
        product_names_list.append(link.text)
    a=len(product_urls_list)
        
    #Product Price
    prices=soup.find_all("span", attrs={'class':"a-price-whole"})
    for p in range(a):
        product_prices_list.append(prices[p].text.strip("."))
        
    #Product Rating and Number of Reviews
    elements=soup.find_all("div", attrs={"class":"a-row a-size-small"})
    for element in elements:
        e=element.find_all("span")
        product_ratings_list.append(e[0].get('aria-label'))
        product_reviews_list.append(e[3].get('aria-label'))
   
    #Product ASIN
    product_ASIN_list= get_ASIN(soup)
    #product_manufacturer_list= get_manufacturer(soup)
    
   
    
    d={'product_url':product_urls_list,
       'product_name': product_names_list,
       'product_price': product_prices_list,
       'product_rating': product_ratings_list,
      'product_no_of_reviews':  product_reviews_list,
      'product_ASIN': product_ASIN_list,
      }

  
    '''
    for i in range(len(product_urls_list)):
        new_webpage= requests.get(product_urls_list[i], headers=headers)
        new_soup=BeautifulSoup(new_webpage.content, 'html.parser')
        
        d['product_description'].append(get_description(new_soup))
        d['product_details'].append(get_details(new_soup))
        d['product_manufacturer'].append(get_manufacturer(new_soup))
        time.sleep(86400)
    '''
    amazon_df=pd.DataFrame.from_dict(d)
    amazon_df['product_url'].replace('',np.nan,inplace=True)
    amazon_df = amazon_df.dropna(subset=['product_url'])
    amazon_df.to_csv("amazon_product_data.csv", header=True, index=False)
   
    
    

In [263]:
amazon_df

,product_url,product_name,product_price,product_rating,product_no_of_reviews,product_ASIN
0,https://www.amazon.in/Skybags-Brat-Black-Casua...,Skybags Brat Black 46 Cms Casual Backpack,"2,009",4.1 out of 5 stars,"5,521",B097L4GWB3
1,https://www.amazon.in/Half-Moon-Backpack-Lugga...,Half Moon Large 37L Laptop Bag Backpack for me...,"3,219",3.9 out of 5 stars,"3,442",B07FZN5K3N
2,https://www.amazon.in/Number-Backpack-Compartm...,FUR JADEN Anti Theft Number Lock Backpack Bag ...,649,4.0 out of 5 stars,"5,601",B08Z1HHHTD
3,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,POLICE Backpack for Men Casual Laptop Bag Offi...,849,4.1 out of 5 stars,"5,851",B09VCLZ3K4
4,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,FUR JADEN Pro Series Smart Tech Anti-Theft Lap...,679,4.6 out of 5 stars,99,B09VTDMRY7
5,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,COSMUS Tuxer 38L Large Laptop Backpack With 3 ...,"1,399",4.1 out of 5 stars,314,B09CQ5T3LN
6,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Half Moon Vision Water Resistant 15.6 inch Lap...,"3,499",4.7 out of 5 stars,14,B0C28QQMFJ
7,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Arctic Hunter Backpack for Men Stylish 15.6-in...,"1,692",4.6 out of 5 stars,34,B09HCJKKF4
8,https://www.amazon.in/TRUE-HUMAN-Anti-Theft-ba...,TRUE HUMAN EMPEROR® Anti-Theft backpack With U...,599,3.7 out of 5 stars,"2,463",B0BYW5SH4L
9,https://www.amazon.in/Half-Moon-Waterproof-Bac...,Half Moon 35 Ltrs Water Resistant 15.6 inch La...,"3,419",3.9 out of 5 stars,"16,955",B0B6FRHV7C
